# 事前準備

訓練データの作成（[参考](https://dev-partner.i-pro.com/space/TPFAQ/1007060562/%E3%82%A2%E3%83%8E%E3%83%86%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E3%83%84%E3%83%BC%E3%83%AB%E3%80%8ElabelImg%E3%80%8F%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9FAI%E3%83%A2%E3%83%87%E3%83%AB%E4%BD%9C%E6%88%90)）

In [ ]:
# YOLOアプリのインストール
!git clone https://github.com/ultralytics/yolov5

# pythonパッケージのインストール
!pip install -r yolov5/requirements.txt

# 課題1

別途支給するテスト用画像(136枚)から、ナンバープレートを検出するプログラムを開発してください。

#### 目標

ナンバープレートを検出する学習モデルの生成

#### 手順

##### 1. 訓練用データを配置する

画像(.jpg)とアノテーション(.txt)のデータセットを訓練用(train)と評価用(val)に分割して本ディレクトリに配置する。


```
datasets
├　images
│　├　train
│　│　└　画像(.jpg)
│　└　val
│　 　└　YOLO形式(.txt)
└　labels
 　├　train
 　│　└　画像(.jpg)
 　└　val
 　 　└　YOLO形式(.txt)
```

##### 2. 訓練する

直下セル実行

重みファイルがyolov5/runs/train/license_plate_detector/weight/に生成される。

In [ ]:
# yoloアプリの訓練を実行する
!python yolov5/train.py --img 640 --batch 16 --epochs 100 --data dataset.yml --weights yolov5/yolov5m.pt --name license_plate_detector

# 課題2

検出したナンバープレートにBounding Boxを描画した画像を出力してください。

#### 目標

訓練済みモデルを使用したBounding Boxの推定

元画像にBounding Boxを描画

#### 手順

##### 0. 課題1のコマンドを実行して、モデルを訓練する

##### 1. テストデータを配置する

Bounding Boxを描画したい画像を配置

```
tests
└　input
 　└　画像(.jpg)
```

##### 2. Bounding Boxを描画する

直下セル実行

出力先のディレクトリ

```
tests
└　output
 　└　画像(.jpg)
```

In [ ]:
import math
import torch
import cv2
import glob
import os

# 訓練済みモデルのインスタンス化
model = torch.hub.load(
    "yolov5", # モデルを読み込むディレクトリ
    "custom", # モデルの名前
    path="yolov5/runs/train/license_plate_detector/weights/best.pt", # 重みファイルのパス
    source="local" # gitの公開モデルではなく、ローカルのモデルを使用する
)

# 検出しきい値
model.conf = 0.5

# データアクセス
input_img_data_path = "tests/input"
output_img_data_path = "tests/output"

# 各画像への処理
# Bounding Boxの計算と描画
for data_path in glob.glob("{0}/*".format(input_img_data_path)):
    file_name = os.path.splitext(os.path.basename(data_path))[0]
    img = cv2.imread(data_path)
    result = model(img)
    for row in result.pandas().xyxy[0].itertuples():
        # 画像の上に四角形を重ねる
        cv2.rectangle(
            img, 
            (math.floor(row.xmin), math.floor(row.ymin)), # Bounding Boxの左上座標
            (math.floor(row.xmax), math.floor(row.ymax)), # Bounding Boxの右下座標
            color=(0, 192, 0), # 青クリーム色の枠線
            thickness=3 # 枠線の太さ
        )
    # 新しい画像ファイルの出力
    cv2.imwrite("{0}/{1}.jpg".format(output_img_data_path, file_name), img)

# 課題3

黄色のナンバープレート(軽自動車)の件数を出力してください。

#### 目標

訓練モデルを使用したBounding Boxの推定

Bouding Box内の代表色を取得

ナンバープレートの色の候補からRGBに最も近いものを判定


#### 手順

##### 0. 課題1のコマンドを実行して、モデルを訓練する

##### 1. テストデータを配置する

Bounding Boxを描画したい画像を配置

```
tests
└　input
 　└　画像(.jpg)
```

##### 2. 黄色のナンバープレートを数える

直下セル実行

黄色のナンバープレートの数が出力される。

In [ ]:
### ナンバープレート種別の読み込み

import json

category_file = "license_plate_category/category.json"

with open(category_file, "r") as f:
    categories = json.load(f)

"""
ナンバープレートのカテゴリを判定する
@param a_score LAB空間色A値
@param b_score LAB空間色B値
@return カテゴリのインデックス
"""
def judge_license_plate_category(a_score, b_score):
    min_ix = -1 # 最小ユークリッド距離を記録した[配列のインデックス]
    min_euclidean_distance = float('inf') # 
    for ix, category in enumerate(categories):
        # 各種別の代表色とのユークリッド距離
        euclidean_distance = (category['a_score'] - a_score) ** 2 + (category['b_score'] - b_score) ** 2
        if euclidean_distance < min_euclidean_distance:
            # 最小のユークリッド距離ならば記録
            min_euclidean_distance = euclidean_distance
            min_ix = ix
    return min_ix

def show_category_name(ix):
    return categories[ix]["name"]

In [ ]:
### 全画像のナンバープレート種別の推定

import torch
import math
import numpy as np


# 訓練済みモデル
model = torch.hub.load(
    "yolov5", # モデルを読み込むディレクトリ
    "custom", # モデルの名前
    path="yolov5/runs/train/license_plate_detector/weights/best.pt", # 重みファイルのパス
    source="local" # gitの公開モデルではなく、ローカルのモデルを使用する
)

# 検出しきい値
model.conf = 0.5

# データアクセス
input_img_data_path = "tests/input"
output_img_data_path = "tests/output"

# 各ナンバープレートのカテゴリ
plate_categories_record = []

# Bounding Box計算とBox中心のRGB推定
for data_path in glob.glob("{0}/*".format(input_img_data_path)):
    print(os.path.basename(data_path))
    img = cv2.imread(data_path)
    result = model(img) # Bounding Boxの計算

    # LAB空間色としてndarray化
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    for row in result.pandas().xywh[0].itertuples():
        # ナンバープレート幅・高さの1/8の領域のくり抜き
        x0 = math.floor(row.xcenter - row.width / 8)
        y0 = math.floor(row.ycenter - row.height / 8)
        x1 = math.floor(row.xcenter + row.width / 8)
        y1 = math.floor(row.ycenter + row.height / 8)
        extract_area = img_lab[y0:y1, x0:x1]

        # 領域の平均の色を取得し、代表色とする
        _, a_score, b_score = np.mean(extract_area, axis=(0, 1))

        # 代表色からナンバープレートの種類を推定する
        plate_category = judge_license_plate_category(a_score, b_score)
        plate_categories_record.append(plate_category)
        print(show_category_name(plate_category))
    print("")

In [ ]:
### ナンバープレート種別の集計
### 結果の出力

dict = {}

for category in categories:
    dict.setdefault(category["name"], 0)

for ix in plate_categories_record:
    category_name = categories[ix]["name"]
    dict[category_name] = dict[category_name] + 1

print("白色のナンバープレートの数：{0}".format(dict["normal"]))
print("黄色のナンバープレートの数：{0}".format(dict["light"]))
print("緑色のナンバープレートの数：{0}".format(dict["normal_business"]))